In [17]:
# Cell 1: Imports
import requests
import os
from dotenv import load_dotenv

In [11]:
# Cell 2: Setup API
load_dotenv()
USDA_API_KEY = os.getenv("USDA_API_KEY")
USDA_BASE_URL = os.getenv("USDA_BASE_URL")

def search_food(food_name, api_key):
    """Search for food in USDA database"""
    url = f"{USDA_BASE_URL}/foods/search"
    params = {
        'api_key': api_key,
        'query': food_name,
        'pageSize': 5,
        'dataType': ['Survey (FNDDS)']  # Standard Reference foods
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [13]:
# Cell 3: Test API
result = search_food("pizza", USDA_API_KEY)

if result:
    print(f"Found {result['totalHits']} results\n")
    
    for food in result['foods'][:3]:  # Show first 3
        print(f"Food: {food['description']}")
        print(f"FDC ID: {food['fdcId']}")
        
        # Get calories
        nutrients = food.get('foodNutrients', [])
        for nutrient in nutrients:
            if nutrient['nutrientName'] == 'Energy':
                calories = nutrient['value']
                unit = nutrient['unitName']
                print(f"Calories: {calories} {unit}\n")
                break

Found 102 results

Food: Dessert pizza
FDC ID: 2708008
Calories: 204 KCAL

Food: Mexican pizza
FDC ID: 2708507
Calories: 249 KCAL

Food: Pizza rolls
FDC ID: 2708686
Calories: 345 KCAL



In [14]:
# Cell 4: Get detailed food info
def get_food_details(fdc_id, api_key):
    """Get detailed nutrition info for specific food"""
    url = f"{USDA_BASE_URL}/food/{fdc_id}"
    params = {'api_key': api_key}
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    return None

In [15]:
# Cell 5: Extract calorie info
def get_calories(food_name, api_key):
    """Simple function to get calories for food name"""
    search_result = search_food(food_name, api_key)
    
    if not search_result or not search_result['foods']:
        return None
    
    # Get first result
    food = search_result['foods'][0]
    food_name = food['description']
    
    # Find calorie value
    for nutrient in food.get('foodNutrients', []):
        if nutrient['nutrientName'] == 'Energy':
            return {
                'food_name': food_name,
                'calories': nutrient['value'],
                'unit': nutrient['unitName']
            }
    
    return None

In [16]:
# Test it
calorie_info = get_calories("banana", USDA_API_KEY)
print(calorie_info)
print(get_food_details(2708686, USDA_API_KEY))

{'food_name': 'Banana, baked', 'calories': 161, 'unit': 'KCAL'}
{'foodClass': 'Survey', 'description': 'Pizza rolls', 'foodNutrients': [{'type': 'FoodNutrient', 'id': 34350789, 'nutrient': {'id': 1003, 'number': '203', 'name': 'Protein', 'rank': 600, 'unitName': 'g'}, 'amount': 9.19}, {'type': 'FoodNutrient', 'id': 34350790, 'nutrient': {'id': 1004, 'number': '204', 'name': 'Total lipid (fat)', 'rank': 800, 'unitName': 'g'}, 'amount': 10.5}, {'type': 'FoodNutrient', 'id': 34350791, 'nutrient': {'id': 1005, 'number': '205', 'name': 'Carbohydrate, by difference', 'rank': 1110, 'unitName': 'g'}, 'amount': 53.4}, {'type': 'FoodNutrient', 'id': 34350792, 'nutrient': {'id': 1008, 'number': '208', 'name': 'Energy', 'rank': 300, 'unitName': 'kcal'}, 'amount': 345}, {'type': 'FoodNutrient', 'id': 34350793, 'nutrient': {'id': 1018, 'number': '221', 'name': 'Alcohol, ethyl', 'rank': 18200, 'unitName': 'g'}, 'amount': 0.0}, {'type': 'FoodNutrient', 'id': 34350794, 'nutrient': {'id': 1051, 'number'